<a href="https://colab.research.google.com/github/lidar532/ppkgeotag/blob/2020-0222-dev/cwwppkgeotaglib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title
# -*- coding: utf-8 -*-
"""cwwppkgeotaglib.ipynb
 C. W. Wright https://github.com/lidar532/  (C) 2018, 2019, 2020

 Library of functions for generating interpolated photo locations
 from a PPK trajectory file, a file of precision high accuracy photo
 flash events, and a file of photo names with cooresponding EXIF
 time stamps.
"""
asof = '2020-0817-0810'
global ppk
import sys
import calendar
import datetime
import os
import platform
import re
import urllib
import matplotlib as mp
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from collections import OrderedDict
from pathlib import Path, PureWindowsPath
import pathlib
from IPython.core.display import HTML, display
from matplotlib import colors
from pandas.plotting import register_matplotlib_converters


try:
  import google.colab
  IN_COLAB = True
  my_platform = 'colab'
except:
  IN_COLAB = False
  my_platform = 'unknown'

# Install pyproj package.
try:
    import pyproj
    print('pyproj module loaded.')
except:
    if 'pyproj' in sys.modules:
        print('Pyproj found. Loaded.')
    else:
        print('Installing pyproj module.')
        !pip install pyproj

from pyproj import Proj

register_matplotlib_converters()

# Changelog
# 	2019-0802 -WW
#		Added ability to estimate the hours and seconds time offset values to fix EXIF data.  It's based on the
#	    last flash and last exif which seem to be more reliable than the first set.
#       
#       Changed load_exif to skip the first picture since it conld be either a header row or a picture.  When it is a header row
#       it causes the time plot to give unusable results. 

# For interactive 
# %matplotlib notebook

# Settings the user can or must set
ppk_user_settings = { 
    'Photos_to_Skip'                : 0,
    'Accuracy_Scale_Factor'        	: 2.0,
    'Accuracy_Offset_Meters' 		    : 0.05,
	'PixPos_Directory'				        : "",
    'Exif_File_Name'                : "",
    'Trajectory_file_Name'          : "",
	'Trajectory_GPS_to_UTC_Time_difference' : 0,
    'Flash_Events_file_Name'        : "",
    'EXIF_drift_correction_seconds' : 0,
    'EXIF_Offset_from_UTC_Hours'    : 0,
    'Base_station_ID'               : "",
    'User_Notes'                    : "",
    'Generate_Output_File'          : "No",
    'Plot_Times'                    : True,
    'Show_File_Stats'               : True,
    'Show_Flash_event_Distribution' : False,
    'Show_XYZ_Std_Devs'             : False,
    'Show_Photo_Location_Plan_View' : False,
    'Show_Photo_Elevations'         : False,
    'Debugging_Output'              : "None"
}

ppk = { 'ppk_flash_sync_version' : f'cwwppkgeotaglib.py asof {asof}',   # Configure ppk variables
		'Notebook Program'		 : 'Not Set',
        'run_error'              : '',
        'run'                    : True,
        'debug'                  : 0,
        'Zulu_offset_hours_exif' : 4.0,
        'exif_offset_seconds'    : -8.0,
        'GPS_base_station_name'  : '',
        'run_time_date'          : '',
        'ofn'                    : '',
        'traj_root_name'         : '',
        'npix'                   : 0,
        'nflash'                 : 0,
        'output_records'         : 0,
        'output_record_sep'      : ',',
        'gps_leap_seconds'       : 0,
        'sec_offset_str'         : 's0',
        'hours_offset_str'       : 'h0',
        'exif_df'                : ''

      }

def gen_output_file_name( ppk, ppk_user_settings):
  """Computes an output filename based on the trajectory file plus the EXIF
  clock offsets in hours and seconds.  The computed filename is returned and
  and placed in ppk['ofn'].
  """
  currentDT = ppk['run_time_date']
  ppk['sec_offset_str']    = 's'+str(ppk_user_settings['EXIF_drift_correction_seconds'])
  ppk['hours_offset_str']  = 'h'+str(ppk_user_settings['EXIF_Offset_from_UTC_Hours'])
  ppk['traj_root_name'] = os.path.splitext(os.path.basename(ppk_user_settings['Trajectory_file_Name'] ))[0]
  ofn = ppk_user_settings['PixPos_Directory'] +'/'+\
    ppk['traj_root_name']+\
    '-'+'h'+str(ppk_user_settings['EXIF_Offset_from_UTC_Hours'])+\
    's'+str(ppk_user_settings['EXIF_drift_correction_seconds'])+\
    '-pixpos.txt'
  ppk['ofn'] = ofn
  return ofn
  
def check_input_files( ppk, ppk_user_settings ):
  """Checks the input files found in ppp_user_settings[] for existence.  
  If they do not exist, it sets ppk['run'] to False.
  """
  if ppk['debug'] > 0:  print('check_input_files()')
  currentDT = ppk['run_time_date'] = datetime.datetime.now()  # Get current time/date to mark running this script today
  ppk['run'] = True
  ppk['ofn'] = "Not Set"
  if ppk['debug'] > 0:
    print("\n    Base Station: ", ppk['GPS_base_station_name'],
          "\n", os.path.isfile( ppk_user_settings['Exif_File_Name'] ), " Exif File:", ppk_user_settings['Exif_File_Name'],
          "\n", os.path.isfile( ppk_user_settings['Trajectory_file_Name'] ),   "  PPK File:", ppk_user_settings['Trajectory_file_Name'], 
          "\n", os.path.isfile( ppk_user_settings['Flash_Events_file_Name']), " Flash File:",  ppk_user_settings['Flash_Events_file_Name'],
          '\n Output Filename:', ppk['ofn'], )
    
  if os.path.isfile( ppk_user_settings['Exif_File_Name'] ) == False:
      ppk['run']  = False
      ppk['run_error'] = f"Exif File not found: {ppk_user_settings['Exif_File_Name']}"
      return ""
  if os.path.isfile( ppk_user_settings['Trajectory_file_Name'] )   == False:
      ppk['run'] = False 
      ppk['run_error'] = f"Trajectory File not found: {ppk_user_settings['Trajectory_file_Name']}"
      return ""
  if os.path.isfile( ppk_user_settings['Flash_Events_file_Name'] ) == False:
      ppk['run'] = False
      ppk['run_error'] = f"Flash Events File not found: {ppk_user_settings['Flash_Events_file_Name']}"
      return ""  
  if ppk['debug'] > 0:        
      print('\n', ppk_user_settings['EXIF_drift_correction_seconds'],   " Camera clock drift correction\n",
            ppk_user_settings['EXIF_Offset_from_UTC_Hours']," Camera offset to UTC\n")
  return ppk['ofn']


# Test code for the cell above goes here.
if __name__ == '__main__':
  ;
def format_single_time(x):
  # s = print(x.strftime("%b %d %Y %H:%M:%S\t"), end="")     
  return x.strftime("%b %d %Y %H:%M:%S")

def show_times( exif_df, ppk_trj, flashs):
  first_photo_time = exif_df['Correct_exif_Ztime'].min()
  last_photo_time  = exif_df['Correct_exif_Ztime'].max()
  first_trj_time = ppk_trj['Ztime'].min()
  last_trj_time  = ppk_trj['Ztime'].max()
  first_flash_time = flashs['Flash_Ztime'].min()
  last_flash_time = flashs['Flash_Ztime'].max()

  print( "     Corrected Photo Exif times")
  print( "     Start Time                  Stop Time")
  print( "     ", format_single_time(first_photo_time),  format_single_time(last_photo_time),   " Photo EXIF times" )
  print( "     ", format_single_time(first_flash_time),  format_single_time(last_flash_time),   " Flash event Times")
  print( "     ", format_single_time(first_trj_time),    format_single_time(last_trj_time),     " Trajectory Times" )
  return


In [ ]:

####   See encoding option to fix reading from exiftool written from window 10 > file.    
def load_exif( ppk, ppk_user_settings ):
  """Load the EXIF datafile to a pandas dataframe ( exif_df )
       1) Load in the exif information from a file
       2) Convert the date/time string to actual time and add as a column to the data frame
     ppk_user_settings['EXIF_Offset_from_UTC_Hours']= pd.to_timedelta( ppk_user_settings['EXIF_Offset_from_UTC_Hours'], unit="h")

     Input: ppk, ppk_user_settings['Exif_File_Name']

     Outputs: Sets ppk['run'], ppk['run_error']

     Returns: exif_df pandas dataframe.
  """
  if ppk['debug'] > 0:
      print('load_exif() ')

  exif_df = pd.read_csv( ppk_user_settings['Exif_File_Name'], 
                        names=['pix', 'raw_exif_ymd_hms', 'iso', 'shutter' ],
                        delimiter=',',
                        comment='#',
                        skiprows = 0 )
  exif_df = exif_df[ppk_user_settings['Photos_to_Skip']:]
  # Convert date/time to actual time value and add as a column time
  exif_df['exif_time'] = pd.to_datetime(exif_df['raw_exif_ymd_hms'], format=" %Y:%m:%d %H:%M:%S", errors='coerce')
  exif_df['exif_time'] = pd.to_datetime(exif_df['raw_exif_ymd_hms'], format="%Y:%m:%d %H:%M:%S", errors='coerce')

  # Generate a correct exif_Ztime from the EXIF time stamp corrected for drift and UTC offset
  exif_df['Correct_exif_Ztime'] = exif_df['exif_time'] + \
                                  pd.to_timedelta( ppk_user_settings['EXIF_Offset_from_UTC_Hours'], unit="h")+ \
                                  pd.to_timedelta( ppk_user_settings['EXIF_drift_correction_seconds'], unit="s")
  exif_df['Correct_exif_ZtimeIdx'] = exif_df['Correct_exif_Ztime']
  exif_df = exif_df.set_index('Correct_exif_ZtimeIdx')
  ppk['npix']   = exif_df.shape[0]
  if pd.isnull(exif_df['exif_time'][1]):
    ppk['run'] = False
    ppk['run_error'] = f"exif_df is invalid. The EXIF File format is wrong: {ppk_user_settings['Exif_File_Name']}"
  return exif_df


In [ ]:

def load_ppkca_trj( ppk, ppk_user_settings ):
	fn = pathlib.Path(ppk_user_settings['Trajectory_file_Name'] )
	if fn.exists() == False:
		print(fn," does not exist.")
		ppk['run'] = False
	else:
		try:
			gps_to_utc = ppk_user_settings['Trajectory_GPS_to_UTC_Time_difference']
			print('gps_to_utc:', gps_to_utc )
			ppp_pos = pd.read_csv(ppk_user_settings['Trajectory_file_Name'], skiprows=5, sep='\s+' )
			ppk_data = ppp_pos
			ppk_data['Ztime']       = pd.to_datetime( ppp_pos['YEAR-MM-DD'] +" "+ppp_pos['HR:MN:SS.SS'] ) \
										+ pd.to_timedelta(gps_to_utc, unit='seconds')
			ppk_data['GPS_lat']     =  ppp_pos['LATDD']  + ppp_pos['LATMN']/60.0 + ppp_pos['LATSS']/3600.0
			ppk_data['GPS_lon']     = -( abs(ppp_pos['LONDD']) + ppp_pos['LONMN']/60.0 + ppp_pos['LONSS']/3600.0 )
			ppk_data['GPS_nad83h']  = ppp_pos['HGT(m)']
			ppk_data['sdu']         = ppp_pos['SDHGT(95%)'] / 1.95
			ppk_data['ns']          = ppp_pos['NSV']
			ppk_data['Q']           = 6
			ppk_data['Ztime_idx'] = ppk_data['Ztime']
			ppk_data = ppk_data.set_index('Ztime_idx')
			return ppk_data
		except:
			print(f"{ppk_user_settings['Trajectory_file_Name']} does not appear to be a PPP-ca file and pd.read_csv failed." )
			ppk['run'] = False
			ppk['run_error'] = f"{ppk_user_settings['Trajectory_file_Name']} does not appear to be a PPP-ca file and pd.read_csv failed."
			return {}



def load_rtklib_trj( ppk, ppk_user_settings ):
    """
    Loads a .pos "Position file" generated by RTKLIB into the pandas dataframe ppk_trj.  The .pos file
    is expected to contain UTC time and not GPS time.
    """
    
    if ppk['debug'] > 0: print('load_rtklib_trj()')
    ppk_trj = pd.read_csv( ppk_user_settings['Trajectory_file_Name'], 
                          names=['date', 'zhms', 
                                 'GPS_lat', 'GPS_lon', 'GPS_nad83h','Q', 'ns',
                                 'sdn', 'sde', 'sdu', 'sdne', 'sdeu', 'sdun', 'age', 'ratio'],
                          delim_whitespace=True,
                          comment='%',
                          skiprows = 1 )
    # ppk_trj
    ppk_trj['Ztime'] = pd.to_datetime( ppk_trj['date']+' '+ppk_trj['zhms'])    # +pd.Timedelta( ppk_trj['date'])
    ppk_trj['Ztime_idx'] = ppk_trj['Ztime']
    ppk_trj = ppk_trj.set_index('Ztime_idx')

    if ppk['debug'] > 0:
        # ppk_trj[0:4][['date','zhms','GPS_lat','GPS_lon','GPS_nad83h','Q','ns']]    
        display(ppk_trj[1:5])
    return ppk_trj



In [ ]:

def load_k706_flash_events( ppk, ppk_user_settings, duplicates='remove' ):
  """
  Load a ComNav K706 or K501 GNSS file containing #MARKTIMEA event records. These contain the 
  flash shoe "Event" time stamp data marks captured when the camera shutter fires.

  Inputs:
    ppk, 
    ppk_user_settings, 
    duplicates=['remove', 'return', 'keep']     # Default is to remove the duplicates.

  Returns:
      flash_stamps_df Pandas data frame.

  """
  if ppk['debug'] > 0:
      print('load_k706_flash_events()')
  cols = ['gps_week', 'sow', 't1', 't2', 'Zulu Offset', 'cksum' ]
  flash_stamps_list = []
  gga_list = []
  flash_stamps_f = open( ppk_user_settings['Flash_Events_file_Name'] )
  rx = re.compile(r'#MARKTIMEA,*')
  #rxgga = re.compile(r'\$GPGGA,*')
  for line in flash_stamps_f:
      w = rx.findall(line)
  #    wgga = rxgga.findall(line)
      if w :
          s = line.split(";")
          s = s[1].split(',')
          flash_stamps_list.append( s )
  #    if wgga :
  #        s = line.split(',')
  #        gga_list.append( s )

  gga_df = pd.DataFrame.from_records(gga_list)
  # , parse_dates=True, date_parser=nmea2datetime

  ##  unit of the arg (D,h,m,s,ms,us,ns) denote the unit, which is an integer/float number
  flash_stamps_df = pd.DataFrame.from_records( flash_stamps_list, columns=cols )
  ppk['gps_leap_seconds'] = flash_stamps_df['Zulu Offset'].min()
  flash_stamps_df['gps_week'   ] = pd.to_numeric(flash_stamps_df['gps_week'])

  # Using the GPS week value, compute the days offset to pandas time.        
  gps_week_offset = pd.to_timedelta( (flash_stamps_df['gps_week'][1]+52*10)*7+17, unit="D")

  flash_stamps_df['gps_day'    ] = pd.to_timedelta(flash_stamps_df['gps_week']*7, unit = 'D' )
  flash_stamps_df['Zulu Offset'] = pd.to_numeric(flash_stamps_df['Zulu Offset'])
  flash_stamps_df['sow'        ] = pd.to_numeric(flash_stamps_df['sow'])
  flash_stamps_df['gps_sow'    ] = pd.to_timedelta(flash_stamps_df['sow'], unit='s')
  flash_stamps_df['Flash_Ztime'] = pd.to_datetime(flash_stamps_df['sow']+ flash_stamps_df['Zulu Offset'], unit='s')
  flash_stamps_df['Flash_Ztime'] = flash_stamps_df['Flash_Ztime'] + gps_week_offset
  flash_stamps_df['Flash_ZtimeIdx'   ] = flash_stamps_df['Flash_Ztime']
  flash_stamps_df['t1'         ] = pd.to_numeric(flash_stamps_df['t1'])
  flash_stamps_df['t2'         ] = pd.to_numeric(flash_stamps_df['t2'])
  flash_stamps_df['Seconds_Offset'] = int(np.timedelta64( ppk_user_settings['EXIF_drift_correction_seconds'] ).astype(int) / 1e6)
  

  ppk['nflash'] = flash_stamps_df.shape[0]
  flash_stamps_df = flash_stamps_df.set_index('Flash_ZtimeIdx')
  if ppk['debug'] > 0:
      print("GPS Leap Seconds: ", ppk['gps_leap_seconds'], "\n")
      print(ppk['nflash'], " Flash Time Stamps found in File: ", ppk_user_settings['Flash_Events_file_Name'] )
      print(  ppk['npix'], "       Photos found in EXIF file: ", ppk_user_settings['Exif_File_Name'] )
  if ppk['debug'] > 1:
      display( flash_stamps_df[0:10]  )

  if duplicates == 'keep':     # Return the actual dataset.
    return flash_stamps_df
  elif duplicates == 'remove':
    return flash_stamps_df.drop_duplicates(subset='Flash_Ztime', keep='first')
  elif duplicates == 'return':    # Return only the duplicates
    return flash_stamps_df[flash_stamps_df.duplicated( subset='Flash_Ztime', keep='first')]




In [ ]:
#@title Test flash_events_file {form-width: "25%"}
# The following used to test with 2020-0802 NC dataset.
if __name__ == '__main__':
  Flash_Events_file_Name = "/content/PPK_Sample_data/2019-0311-Fla-LarrySandersPark/aircraft-solo/gps/GP184856.TXT" #@param {type:"string"}
  ppk_user_settings['Exif_File_Name'] = 'Flash_Events_file_Name'
  flash_stamps_df = load_k706_flash_events( ppk, ppk_user_settings, duplicates='remove' )
  

  flash_duplicates = load_k706_flash_events( ppk, ppk_user_settings, duplicates='return' )
  duplicates_found = flash_duplicates.count()["Flash_Ztime"]
  print(f'Duplicates Found:{duplicates_found}')
  if duplicates_found:
    display( flash_duplicates )
    



In [ ]:


def interpolate_flash_positions(ppk, ppk_user_settings, exif_df, ppk_trj, flash_stamps_df):
  # Step 7.    Interpolate the lat/lon/elevation for each record in flash_stamps_df, also join other data from ppk file.
  # Interpolate the lat, Lon, and Altitude from the trajectory to the flash time stamps
  #trig_lat = np.interp( flash_stamps_df['Ztime'], ppk_trj['Ztime'], ppk_trj['lat'])

  if ppk['debug'] > 0:
      print('interpolate_flash_positions()')

  # Save a copy of flash_stamps_df to Y so we can restore it later and also make a copy to
  # Xflash_stamps_df.
  Y = flash_stamps_df
  Xflash_stamps_df = flash_stamps_df

  # Round to the nearest second on the index column
  Xflash_stamps_df.index = flash_stamps_df.index.round('1s')
  Xflash_stamps_df       = flash_stamps_df.join( ppk_trj, how='inner')

  ee = datetime.datetime(1970, 1, 1)
  x0 =  flash_stamps_df['Flash_Ztime'] - ee
  x1 =  ppk_trj['Ztime'] - ee
  y0 =  ppk_trj['GPS_lat']

  xx0 = x0 / np.timedelta64(1, 'us')
  xx1 = x1 / np.timedelta64(1, 'us')

  ppk_trj.index = ppk_trj.index.round('10us')

  # Interpolate lat, long, and elevation from the trajectory into the flash data frame
  flash_stamps_df['PIX_lat'   ] = np.interp( xx0, xx1, ppk_trj['GPS_lat'   ])
  flash_stamps_df['PIX_lon'   ] = np.interp( xx0, xx1, ppk_trj['GPS_lon'   ])
  flash_stamps_df['PIX_nad83h'] = np.interp( xx0, xx1, ppk_trj['GPS_nad83h'])

  # Merge in the other columns from the ppk_trj to pickup the stats, number of sats, and such
  flash_stamps_df = flash_stamps_df.join( ppk_trj, how='inner')

  # display(flash_stamps_df[30:34][['Flash_Ztime', 'PIX_lat', 'PIX_lon', 'PIX_nad83h','GPS_lat','GPS_lon','GPS_nad83h', 'Q', 'ns', 'sdu', 'age', 'ratio']] )
  Xflash_stamps_df = flash_stamps_df
  flash_stamps_df = Y

  # Add in the camera data, iso, shutter time, etc. from the exif dataframe into te output record.
  output_data = exif_df.join( Xflash_stamps_df, how='inner' )
  ppk['output_records'] = output_data['pix'].count()
  output_data['Seconds_Offset'] = ppk_user_settings['EXIF_drift_correction_seconds']
  output_data['Accuracy_Z'] = output_data['sdu']*ppk_user_settings['Accuracy_Scale_Factor']+ppk_user_settings['Accuracy_Offset_Meters']
  print('Output_data records found:', output_data['pix'].count())
  return output_data

def generate_output_header(ppk, ppk_user_settings,  odf ):
  currentDT = datetime.datetime.now()
  print( '# C. W. Wright https://github.com/lidar532  Generated: ', str(currentDT),
        '\n# Generating Program:', ppk['Notebook Program'],'  Library Version:', ppk['ppk_flash_sync_version'],
        '\n# ',ppk_user_settings['User_Notes'],
        '\n#  Trajectory File:', ppk_user_settings['Trajectory_file_Name'],
        '\n#        EXIF File:', ppk_user_settings['Exif_File_Name'], "  ", ppk['npix'], " Photos  ",
        ' Applied Clock Drift Correction:', ppk_user_settings['EXIF_drift_correction_seconds'], ' Seconds.  EXIF UTC Offset: ', ppk_user_settings['EXIF_Offset_from_UTC_Hours'],
        '\n# Flash Event File:', ppk_user_settings['Flash_Events_file_Name'], "  ", ppk['nflash'], ' Flash Events.  Base Station: ',
        ppk['GPS_base_station_name'], file=odf )

def generate_output_file( ppk, ppk_user_settings,  output_data ):
  # Generate the output file suitable for input to Agisoft Photoscan
  # Wrte the output data file
  if ppk['output_records'] == 0 :
    ppk['run'] == False
    display(HTML("<h1 style=""color:Red""><b> WARNING:  The photos do not overlap the Flash Events and no output file<br>"+
                 "can be generated. Make sure your trajectory is in UTC and not GPS Time,<br>"+
                 " and your exif_offset_seconds.</b></h1>"))
                 
    #print('*************************************************************************************************************')
    #print("* WARNING:  The photos do not overlap the Flash Events and no output file can be generated.                 *")
    #print("*            Make sure your trajectory is in UTC and not GPS Time, and your exif_offset_seconds is correct. *")
    #print('*************************************************************************************************************')
    return ''
  gen_output_file_name( ppk, ppk_user_settings)
  if ppk['debug'] > 0:
      print('generate_output_file()', ppk['ofn'])
      print('Computed output file name: ', ppk['ofn'] )
  odf = open(ppk['ofn'], 'w')

  generate_output_header(ppk, ppk_user_settings,  odf )
  output_data.to_csv( odf,
                  sep= '\t', index=False,
                  columns = [ 'pix','PIX_lat','PIX_lon','PIX_nad83h', 'Accuracy_Z', 'Flash_Ztime','raw_exif_ymd_hms', 
                      'GPS_nad83h','GPS_lat','GPS_lon','Q', 'ns', 'sdu', 'ratio', 'gps_sow', 'Seconds_Offset' ],
                  header = [
                      'Photos.DNG                         ', 
                      'PIX_Lat      ',
                      'PIX_Lon    ',
                      'PIX_Elev    ',
					  'Accuracy_Z    ',
                      'raw_exif_ymd_hms',
                      'Flash_Event_UTC_DATE_Time    ',
                      'GPS_Elevation',
                      'GPS_Lat',
                      'GPS_Lon',
                      'Q',
                      'N_Sats',
                      ' ZStdev    ',
                      ' Ratio',
					            'GPS_Time',
								'Seconds_Offset'
                  ],
                  float_format='%10.8f',
                 index_label = '# Date     UTC Time'
                )

  print( '# Footer ', file=odf)
  odf.close()
  print('Output File closed')
  return 'ok'

def extract_rtklib_settings( fn, ppk_user_settings ):
  # Sample input line
  # % obs start : 2019/02/25 16:49:39.5 UTC (week2042 146997.5s)
  traj_comments = OrderedDict()
  with open( ppk_user_settings['Trajectory_file_Name']) as ppk_traj_fdx:
      line = ppk_traj_fdx.readline()
      cnt = 1
      imp_file_cnt = 0
      while line:
          cnt = cnt + 1
          comment   = re.split("^%", line )                # Split out comments
          if len(comment) == 2:
              comment = re.sub("^ ","", comment[1] )       #
              comment = re.split(" : ", comment )
              if len(comment) == 2: 
                  var = comment[0]
                  var = var.rstrip()                       # Strip trailing spaces from var name
                  if var == 'inp file':                    # 
                      imp_file_cnt = imp_file_cnt + 1
                      var = var+"-"+str(imp_file_cnt)
                  val = comment[1].rstrip()                # strip trailing spaces and newline, Build list
                  traj_comments[var] = val
          if cnt >= 100: break
          line = ppk_traj_fdx.readline()
  return traj_comments

def gen_horizontal_table(cshow, dlist, nspaces, caption="", width='50%'):
  """
  Generates an HTML table string of dict values indexed by cshow.

  Inputs: 
      cshow:   list of indexes, 
      dlist:   dict_list, 
      nspaces: number of white spaces to add to the end
      caption: Set the caption of the table. Optional
      width:   Set the table width in % of the window

  Returns: an html table string
  """
  html_string = '<table width='+width+'<caption>'+caption+'<tr>'
  for x in cshow:
      html_string = html_string +'<th><center>'+x.ljust(nspaces)+'</center></th>'
      # print(x.ljust(nspaces), end='')
  html_string = html_string+'</tr><tr>'
  for x in cshow:
      html_string = html_string+'<td><center>'+dlist[x].ljust(nspaces)+'</center></td>'
      # print(traj_comments[x].ljust(nspaces), end='')
  html_string = html_string +'</tr></table>'
  return html_string

def show_rtklib_settings( traj_comments ):
  """
  Display the values and settings extracted from the RTKlib position file as an HTML table.
  """
  width = '55%'     # Table width in % of screen
  cshow = (  'pos mode',  'freqs',  'solution',  'elev mask',  'dynamics',  'tidecorr' )
  s = gen_horizontal_table( cshow, traj_comments, 1, caption="RTKLib Settings", width=width  )

  cshow = (  'ionos opt', 'tropo opt',  'ephemeris',  'navi sys',  'amb res',  'val thres' )
  s = s+gen_horizontal_table( cshow, traj_comments, 1, width=width )

  cshow = ( 'program',   'antenna1' )
  s = s+gen_horizontal_table( cshow, traj_comments, 1, width=width )

  cshow = ( 'antenna2',  'ref pos'  )
  s = s+gen_horizontal_table( cshow, traj_comments, 1, width=width )

  cshow = ( 'inp file-1', 'inp file-2', 'inp file-3', 'inp file-4', 'inp file-5', 'inp file-6', 'inp file-7'  ) 
  s = s + "<table><caption>RTKlib GPS RINEX Rover, Base, Nav, Ephemris, Clock Files</caption><tr>\n"
  for a in cshow:
      if a in traj_comments: 
          s = s + "<th>"+a+"</th><td style=text-align:left;>"+traj_comments[a]+"</td></tr>"
  s = s + "</table>"
  display(HTML(s))
  return

def show_file_stats(ppk_user_settings, ppk_trj_df, output_df ):
  """
  Display stats on the input files, and a graphic showing the distribution of the times in in put files. 
  """
   # Disable decoding trajectory header for now.
  ### traj_comments = extract_rtklib_settings( ppk_user_settings['Trajectory_file_Name'] )
  ### show_rtklib_settings(  traj_comments )					
  first_photo_time = exif_df['Correct_exif_Ztime'].min()
  last_photo_time  = exif_df['Correct_exif_Ztime'].max()
  first_trj_time   = ppk_trj_df['Ztime'].min()
  last_trj_time    = ppk_trj_df['Ztime'].max()
  first_flash_time = flash_stamps_df['Flash_Ztime'].min()
  last_flash_time  = flash_stamps_df['Flash_Ztime'].max()
  out_min          = output_df['Flash_Ztime'].min()
  out_max          = output_df['Flash_Ztime'].max()
  
  print(f'last_flash_time: {last_flash_time}  last_photo_time: {last_photo_time}    ')
  tdiff_seconds = (last_flash_time - last_photo_time).total_seconds()
  estimated_offset_hours  = int(round(tdiff_seconds/3600))
  estimated_offset_seconds = round(tdiff_seconds - estimated_offset_hours*3600)
  
  # selecting components of datatime object
  # https://stackoverflow.com/questions/41782920/how-do-i-format-a-pandas-timedelta-object
  cds = exif_offset_seconds = ppk_user_settings['EXIF_drift_correction_seconds']
#  if exif_offset_seconds.components.days < 0 : 
#      cds = exif_offset_seconds.components.seconds - 60
#  else:
#      cds = exif_offset_seconds.components.seconds
  print("                             Executed:", datetime.datetime.now())
  print("Trajectory_GPS_to_UTC_Time_difference:", ppk_user_settings['Trajectory_GPS_to_UTC_Time_difference'] )
  #print("                            Platform:", platform.node(), " User:", os.getlogin(), " Version:", ppk_flash_sync_version) 
  print("                   Generating Program:", ppk['Notebook Program'])
  print("                           User Notes:", ppk_user_settings['User_Notes'])
  print("                       EXIF File Name:", ppk_user_settings['Exif_File_Name'] )
  print("                             PPK Traj:", ppk_user_settings['Trajectory_file_Name'] )
  print("                    Trajectory_Source:", ppk_user_settings['Trajectory_Source'] )
  print("                         Flash Events:", ppk_user_settings['Flash_Events_file_Name'] )
  print("                Output Directory File:", ppk_user_settings['PixPos_Directory'])
  print("           Output Photo Position File:", os.path.basename(ppk['ofn']))
  print("")
  print("              EXIF UTC Camera         ")
  print("Photo   Flash  Offset   Drift   Base")
  print("Count   Events Hours   Seconds Station      Start Time              End Time           Source")
  print("-------------------------------------------------------------------------------------------------------------")
  print(ppk['npix'], '',   
        ppk_user_settings['EXIF_Offset_from_UTC_Hours'],
        cds,
        ppk['GPS_base_station_name'],
        format_single_time(first_photo_time), format_single_time(last_photo_time), "Camera EXIF",
        sep="\t")

  print( '',ppk['nflash'],"\t\t", format_single_time(first_flash_time), format_single_time(last_flash_time), "Flash Events", sep="\t" )
  print( "\t\t\t\t", format_single_time(first_trj_time), format_single_time(last_trj_time), "PPK", sep="\t" )
  if ppk['output_records'] > 0 :
    print( ppk['output_records'],"\t\t\t", format_single_time(out_min), format_single_time(out_max), "Output", sep="\t" )
  else :
    print( '\t \t\t\t\t** No EXIF Photo date/times that matche Flash Events **')
  if ( estimated_offset_hours ):
    display(HTML("<h1 style=background-color:red;color:white;><b>Estimated required EXIF_Offset_from_UTC_Hours: "+str(estimated_offset_hours)+" Hours</b></h1>"))
  if ( estimated_offset_seconds ):
    display(HTML("<h1 style=background-color:red;color:white;><b>Estimated required EXIF_drift_correction_seconds: "+str(estimated_offset_seconds)+" Seconds</b></h1>")) 
  print("------------------------------------------------------------------------------------------------------------")
  return

def gen_q_colors(output_data):    
  # generate an array of colors based on the "Q" of each ppk photo position
  q = output_data['Q']
  q_colors = np.where(        q == 1 , 'green', q )
  q_colors = np.where(q_colors == '2', 'orange', q_colors )
  q_colors = np.where(q_colors == '4', 'blue',   q_colors )
  q_colors = np.where(q_colors == '5', 'red',    q_colors )
  return q_colors

def gen_q_sizes(output_data):
  # adjust the point size based on "Q".  Make Q2, Q3 values larger
  q = output_data['Q']
  q_size = np.where( q == 1,  2.5, q )
  q_size = np.where( q_size == 2, 10, q_size) 
  q_size = np.where( q_size == 4, 40, q_size) 
  q_size = np.where( q_size == 5, 60, q_size)
  return q_size

# plot the distribution of photo event times around each one second mark
def photo_event_time_distro(ppk, ppk_user_settings,  exif_df, flash_stamps_df ):
  # Compute the time difference between the flash time stamps, and the Ricoh clock EXIF values
  #flash_stamps_df.head()
  if ppk['output_records'] == 0:
    return
  if ppk['debug'] > 0:
      print("photo_event_time_distro()")
      print( ppk['nflash'], " Flash Time Stamps found in File: ", ppk_user_settings['Flash_Events_file_Name'])
      print(  ppk['npix'], "      Photos flound in EXIF file: ", ppk_user_settings['Exif_File_Name'], "\n")
  # print("", flash_stamps_df['Ztime'] - exif_df['Ztime'])
  tdifs = (flash_stamps_df['Flash_Ztime'] - exif_df['Correct_exif_Ztime'])  / np.timedelta64(1, 's')
  tdifs = tdifs.where( (tdifs < 9.0) & ( tdifs > -9.0) )

#   %matplotlib inline

  plt.rcParams["figure.dpi"] = 100    
  plt.xlabel('Photo Event time offset from each GPS Second')
  plt.ylabel('Photo Events')
  thist = tdifs.plot( bins=1000,
                          x = 'Seconds',
                          kind='hist',
                          title='Photo Event Time Distribution',
                          figsize = (10,3)
                         )
  plt.show()
  return

def plot_times( exif_df, flash_stamps_df, ppk_trj_df, output_df ):
  """
  Plots the flash event times, the corrected exif times, and the traj times. using matplotlib. Useful
  to check time alignment of the three input files.

  """
#   %matplotlib inline
  title = 'Date / Times within each File'
  x_label = 'Date / Time'
  y_label = 'Event'
  plt.rcParams["figure.dpi"] = 100
  plt.figure(figsize=(15,1) )
  x = pd.to_datetime(exif_df['Correct_exif_Ztime'])
  q_color = gen_q_colors( output_df)
  q_size  = gen_q_sizes(output_df)

  
  x = ppk_trj_df['Ztime']
  y = np.empty( len(x))
  y.fill(1)
  plt.plot_date( x, y,
      markersize = 15.0, color="green", label="GPS Trajectory Times"
  )

  x = exif_df['Correct_exif_Ztime']
  y = np.empty( len(x))
  y.fill(1.0)
  plt.plot_date( x,y,
      markersize = 10.0, color="Black", marker="s", label="Photo Times"
  )
    
  x = flash_stamps_df['Flash_Ztime']
  y = np.empty( len(x))
  y.fill(1.0)
  plt.plot_date( x,y,
      markersize = 4.0, color="Orange", marker="s", label="Flash Event Times"      
  )
  
  plt.xticks(rotation = 35)
  plt.title(title)
  plt.xlabel(x_label)
  #    plt.ylabel(y_label)
  plt.legend(loc=(0.0, 1.0))
  plt.show()
  return

def plot_pix_elevations( ppk, ppk_user_settings,  output_data ):
  """
  Plots the elevations vs sow (Seconds of the week) using matplotlib.

  """
  if ppk['output_records'] == 0:
    return
  if ppk_user_settings['Debugging_Output'] != 'None':
    print('Enter plot_pix_elevations()')
  title = 'PPK Photo Elevations (meters)'
  x_label = 'Seconds of the Week'
  y_label = 'Camera PPK Elevation (meters)'
  plt.rcParams["figure.dpi"] = 100
  q_color = gen_q_colors( output_df )
  q_size  = gen_q_sizes(  output_df )
  output_data.plot.scatter( 
      x='sow', y='PIX_nad83h', 
      s=q_size, 
      c=q_color, 
      figsize=(10,3)
  )
  plt.title(title)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.show()
  if ppk_user_settings['Debugging_Output'] != 'None':
    print('Leave plot_pix_elevations()')
  return

# Plot the camera PPK positions in plan view
def plot_planview( ppk, output_data, proj='meters' ):
  """Plots a plan view of the camera locations using matplotlib.  
  It can plot in UTM, meters, or LatLon degrees.  It uses the computed values
  found in the output_data global variable. The plotted points are colored
  according to the PPK Quality factor. Q=1 is Green, Q=2 is Mustard, Q=5 is Red.
  The quality flag Q and the marker color means: 

  1: Fixed, 2: Float, 4: DGPS, 5: Single

     Parameters
     ----------
     proj = ['latlon', 'utm', 'meters']

     Returns
     -------
     Nothing

     References: http://www.rtklib.com/prog/manual_2.2.2.pdf
  """
  if ppk['output_records'] == 0:
    return
#   %matplotlib inline
  lat = output_data['PIX_lat'].values
  lon = output_data['PIX_lon'].values

  # generate an array of colors based on the "Q" of each ppk photo position
  q = output_data['Q']
  q_colors = np.where(        q == 1 , 'green', q )
  q_colors = np.where(q_colors == '2', 'orange', q_colors )
  q_colors = np.where(q_colors == '4', 'blue',   q_colors )
  q_colors = np.where(q_colors == '5', 'red',    q_colors )

  # adjust the point size based on "Q".  Make Q2, Q3 values larger
  q_size = np.where( q == 1,     2.5, q )
  q_size = np.where( q_size == 2, 10, q_size) 
  q_size = np.where( q_size == 4, 40, q_size) 
  q_size = np.where( q_size == 5, 60, q_size)

  if proj == 'meters' or proj == 'utm':        
      # Compute the UTM zone
      z = utm_zone( lon[0])
      proj_string = '+proj=utm +zone={z:1.0f}, +north +ellps=WGS84 +datum=NAD83 +units=m +no_defs'.format(z=z)
      print(proj_string)
      myProj = Proj( proj_string)
      x, y = myProj( lon, lat)
      if proj == 'meters':
          x = x - int(x.min())
          y = y - int(y.min())
          title = 'PPK Camera Positions in Meters'
          x_label = 'Meters'
          y_label = 'Meters'

      elif proj == 'utm':
          title = 'PPK Camera Positions in UTM Meters'
          x_label = 'UTM Meters Easting'
          y_label = 'UTM Meters Northing'

  elif proj == 'latlon':
      title = 'PPK Camera Positions in Lat / Lon'
      x_label = 'Longitude (Degrees)'
      y_label = 'Latitude (degrees)'
      x = lon
      y = lat

  plt.figure(figsize=(10, 8), dpi=100)
  plt.grid(True, linestyle=':')
  plt.scatter( x, y, s=q_size, c=q_colors )
  plt.title(title)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.show()
  return

# Compute the UTM zone from a single longitude value
def utm_zone( lon ):
  """utm_zone( lon )
  Given the longitude as input, returns the UTM zone number.
  Parameters:
  --------
  Input:   lon   Single longitude value.
  Returns: UTM Zone
  """
  z = int((np.floor((lon + 180.0) / 6 ) % 60) + 1)
  return z

def plot_pix_stdevs( ppk, ppk_user_settings, output_data ):
  """
  Plots (matplotlib) separate Std Devs for Z, Northing, and Easting vs sow (Seconds of the week)
  using matplotlib. The points are colored by the GPS Quality factor.
  Inputs: ppk                The ppk data structure
          ppk_user_settings  The ppk user settings
          output_data        The output_data structure to be plotted.
  Output: A scatter plot of time vs the X,Y and Z StdDevs.  Plot is colorized by GPS Quality factor.  
  """
  if ppk['output_records'] == 0:
    return
  
  if ppk_user_settings['Debugging_Output'] != 'None':
    print('Enter plot_pix_stdevs()')
  height = 2
  title = 'PPK Photo Elevations (meters)'
  x_label = 'Seconds of the Week'
  y_label = 'Camera PPK Elevation (meters)'
  plt.rcParams["figure.dpi"] = 100
  q_color = gen_q_colors( output_data )
  q_size  = gen_q_sizes( output_data )

  title = 'Z PPK Std Dev per Photo '
  x_label = 'Seconds of the Week'
  y_label = 'Z Stdev (meters)'

  x = output_data['sow']
  plt.figure(figsize=(10, height), dpi=100)
  plt.grid(True, linestyle=':')
  plt.scatter( x, output_data['sdu'], s=1, color='blue')
  plt.scatter( x, output_data['sdn'], s=1, color='red')
  plt.scatter( x, output_data['sde'], s=1, color='green')
  plt.title(title)
  plt.xlabel(x_label)
  plt.ylabel(y_label)
  plt.show()
  if ppk_user_settings['Debugging_Output'] != 'None':
    print('Leave plot_pix_stdevs()')
  return

# This is the main functions for users to call to process a dataset.
def process_cww_ppk_files(ppk, ppk_user_settings, plots=True, Debug=0, Gen_output = True ):
  """Process PPK position file, Camera EXIF file, and Photo Time Stamp file and produce an
  output file readable by Agisoft Photoscan.  

  Parameters:
  -----------
  plots = [ True | False ]
  Debug = [ 0, 1, 2]                  0= has no debug output, 1=some debug output, 2=max debug output
  Gen_output = [ True | False ]   True = Generate output file, False causes no output file to be written.
  """
  global output_df, exif_df, ppk_trj_df, flash_stamps_df, run, xdebug, output_file_name
  ppk['run'] = True
  ofn                            =     check_input_files( ppk, ppk_user_settings )
  if ppk['run']: exif_df         =             load_exif( ppk, ppk_user_settings )
  if ppk['run'] and ppk_user_settings['Trajectory_Source'] == 'RTKlib': ppk_trj_df      =   load_rtklib_trj( ppk, ppk_user_settings )
  if ppk['run'] and ppk_user_settings['Trajectory_Source'] == 'PPP-Ca': ppk_trj_df      =   load_ppkca_trj( ppk, ppk_user_settings )  
  if ppk['run']: flash_stamps_df 														=   load_k706_flash_events( ppk, ppk_user_settings )
  if ppk['run']: output_df       														=   interpolate_flash_positions( ppk, ppk_user_settings, exif_df, ppk_trj_df, flash_stamps_df ) 

  if ppk['run'] and ppk_user_settings['Generate_Output_File']=='Yes': 
    err = generate_output_file( ppk, ppk_user_settings, output_df )
  if ppk['run'] and ppk_user_settings['Generate_Output_File'] == 'No':
    output_file_name = "Ouput File is turned off. NO OUTPUT FILE GENERATED"
    display(HTML("<h1><b>"+output_file_name+"</b></h1>"))
    
  if ppk['run'] and ppk_user_settings['Show_File_Stats']: show_file_stats(ppk_user_settings, ppk_trj_df, output_df )
  if ppk['run']:
    if ppk_user_settings['Plot_Times']:                    plot_times( exif_df, flash_stamps_df, ppk_trj_df,  output_df  )
    if ppk_user_settings['Show_Flash_event_Distribution']: photo_event_time_distro( ppk,  ppk_user_settings, exif_df, flash_stamps_df )
    if ppk_user_settings['Show_Photo_Location_Plan_View']: plot_planview(       ppk, output_df )
    if ppk_user_settings['Show_Photo_Elevations']:         plot_pix_elevations( ppk, ppk_user_settings,  output_df )
    if ppk_user_settings['Show_XYZ_Std_Devs']:             plot_pix_stdevs(     ppk, ppk_user_settings,  output_df )
  if ppk['run'] == False: print("Process aborted, No output file was generated.") 
  return

print(ppk['ppk_flash_sync_version'])

def is_main_module():
  return __name__ == '__main__' and '__file__' not in globals()



In [ ]:
#@title Create a softlink to a dataset you are working on.
if __name__ == '__main__':
  !ln -s /content/Gdrive/Missions/NC_Coast /content/NC_Coast

In [ ]:
#@title Mount your Google Drive as /content/Gdrive 
if __name__ == '__main__':
  from google.colab import drive
  !rm -rf /content/sample_data/
  drive.mount('/content/.gdrive')
  !ln -sf /content/.gdrive/My\ Drive/ /content/Gdrive
  print('Your Gdrive is mounted')

# Run the following cell to get sample data for testing from github.

In [ ]:
# Execute only when running as __main__.  Requires linux (colabs only for now)
if __name__ == '__main__':
  ! rm -rf /content/sample_data/
  ! rm -rf /content/PPK_Sample_data/
  ! git clone https://github.com/lidar532/PPK_Sample_data.git
  print('Operation completed.')

# Python Notebook Main program below.

In [ ]:
if __name__ == '__main__' and IN_COLAB:
  print(f"{ppk['ppk_flash_sync_version']}   IN_COLAB:{IN_COLAB}")
  #@title PPK Geo Tagging for the CWW-PPK-v2 GNSS System
  #@markdown #Input Data Files
  Trajectory_Source      = "RTKlib" #@param ["RTKlib", "PPP-Ca"]
  Trajectory_GPS_to_UTC_Time_difference =   0#@param {type:"integer" }
  Exif_File_Name = "/content/PPK_Sample_data/2019-0311-Fla-LarrySandersPark/aircraft-solo/exif/2019-0311-Solo-ckscript-F4-exif.csv" #@param {type:"string"}
  Trajectory_file_Name = "/content/PPK_Sample_data/2019-0311-Fla-LarrySandersPark/aircraft-solo/gps/GP184856-NG15P-FLD7.pos" #@param {type:"string"}
  Flash_Events_file_Name = "/content/PPK_Sample_data/2019-0311-Fla-LarrySandersPark/aircraft-solo/gps/GP184856.TXT" #@param {type:"string"}
  #@markdown ---
  #@markdown #Camera EXIF Time Adjustments
  Photos_to_Skip =  0#@param {type:"integer" }
  EXIF_drift_correction_seconds =  -37#@param {type:"integer" }
  EXIF_Offset_from_UTC_Hours =   4#@param {type:"integer"}
  #@markdown ---
  #@markdown #SfM Accuracy
  #@markdown SfM_Accuracy = (Z Standard Deviation) * Accuracy_Scale_Factor +  Accuracy_Offset_Meters
  Accuracy_Scale_Factor = 2.0 #@param {type:"number"}
  Accuracy_Offset_Meters = 0.05 #@param {type:"number"}

  #@markdown ---
  #@markdown #Output File
  PixPos_Directory = "/content/" #@param {type:"string"}
  Base_station_ID = "test" #@param {type:"string"}
  User_Notes = "Test" #@param {type:"string"}
  Generate_Output_File = "Yes" #@param ["No", "Yes"]

  #@markdown ---
  #@markdown #Stats, Graphs & Plots
  Plot_Times = True #@param {type:"boolean" }
  Show_File_Stats = True #@param {type:"boolean" }
  Show_Flash_event_Distribution = True #@param {type:"boolean" }
  Show_XYZ_Std_Devs = False #@param {type:"boolean"} 
  Show_Photo_Location_Plan_View = False #@param {type:"boolean"}
  Show_Photo_Elevations = False #@param { type: "boolean" }
  #@markdown ---
  Debugging_Output = "None" #@param ["None", "Function EntryExit", ""]
  #@markdown ---

# Run this cell to process the data.
if __name__ == '__main__':
  ppk['Notebook Program'] = f'cwwppkgeotaglib as of {asof}'
  print('\nCWW PPK Geotag Library version: ', ppk['ppk_flash_sync_version'])
  ppk_user_settings['Photos_to_Skip']                   = Photos_to_Skip
  ppk_user_settings['Accuracy_Scale_Factor']            = Accuracy_Scale_Factor
  ppk_user_settings['Accuracy_Offset_Meters']           = Accuracy_Offset_Meters
  ppk_user_settings['PixPos_Directory']                 = PixPos_Directory
  ppk_user_settings['Trajectory_Source']                = Trajectory_Source
  ppk_user_settings['Exif_File_Name']                   = '/' +  Exif_File_Name
  ppk_user_settings['Flash_Events_file_Name']           = '/' + Flash_Events_file_Name
  ppk_user_settings['Trajectory_file_Name']             = '/' + Trajectory_file_Name
  ppk_user_settings['EXIF_drift_correction_seconds']    = EXIF_drift_correction_seconds
  ppk_user_settings['EXIF_Offset_from_UTC_Hours']       = EXIF_Offset_from_UTC_Hours
  ppk_user_settings['Generate_Output_File']             = Generate_Output_File
  ppk_user_settings['Base_station_ID']                  = Base_station_ID
  ppk_user_settings['User_Notes']                       = User_Notes
  ppk_user_settings['Generate_Output_File']             = Generate_Output_File
  ppk_user_settings['Plot_Times']                       = Plot_Times
  ppk_user_settings['Show_File_Stats']                  = Show_File_Stats
  ppk_user_settings['Show_Flash_event_Distribution']    = Show_Flash_event_Distribution
  ppk_user_settings['Show_XYZ_Std_Devs']                = Show_XYZ_Std_Devs
  ppk_user_settings['Show_Photo_Location_Plan_View']    = Show_Photo_Location_Plan_View
  ppk_user_settings['Show_Photo_Elevations']            = Show_Photo_Elevations
  ppk_user_settings['Debugging_Output']                 = Debugging_Output
  ppk_user_settings['Generate_Output_File']             = Generate_Output_File

  process_cww_ppk_files(ppk, ppk_user_settings  )
  if ppk['run'] == False:
    print(f"\n{ppk['run_error']}\n")
